# Accessing Geoserver data

This notebooks tests the access to different Geoserver resources and checks that the different user permissions from Magpie are respected.

In [ ]:
# define some useful variables for following steps
import json
import os
import requests
import uuid

print("Setup configuration parameters...")

VERIFY_SSL = True if 'DISABLE_VERIFY_SSL' not in os.environ else False
HEADERS = {"Accept": "application/json", "Content-Type": "application/json"}

PAVICS_HOST = os.getenv("PAVICS_HOST", "pavics.ouranos.ca")
assert PAVICS_HOST != "", "Invalid PAVICS HOST value."

# Magpie variables
MAGPIE_URL = f"https://{PAVICS_HOST}/magpie"

def get_credentials(var_name):
    value = os.getenv(var_name)
    if not value:
        raise ValueError("Missing test admin credentials `{}` to run tests.".format(var_name))
    return value

TEST_MAGPIE_ADMIN_USERNAME = get_credentials("TEST_MAGPIE_ADMIN_USERNAME")
TEST_MAGPIE_ADMIN_PASSWORD = get_credentials("TEST_MAGPIE_ADMIN_PASSWORD")
TEST_GEOSERVER_ADMIN_USERNAME = get_credentials("TEST_GEOSERVER_ADMIN_USERNAME")
TEST_GEOSERVER_ADMIN_PASSWORD = get_credentials("TEST_GEOSERVER_ADMIN_PASSWORD")

MAGPIE_TEST_USER = os.getenv("MAGPIE_TEST_USER") or "test-user-{!s}".format(uuid.uuid4())
MAGPIE_TEST_PASSWORD = os.getenv("MAGPIE_TEST_PASSWORD") or str(uuid.uuid4())
MAGPIE_NO_PERM_USER = os.getenv("MAGPIE_NO_PERM_USER") or "test-user-no-perm-{!s}".format(uuid.uuid4())
MAGPIE_NO_PERM_PASSWORD = os.getenv("MAGPIE_NO_PERM_PASSWORD") or str(uuid.uuid4())

# Geoserver variables
GEOSERVER_SECURED_SERVICE = "geoserver-secured"
GEOSERVER_SECURED_URL = f"https://{PAVICS_HOST}/twitcher/ows/proxy/{GEOSERVER_SECURED_SERVICE}"
GEOSERVER_ADMIN_URL = f"https://{PAVICS_HOST}/geoserver/rest"

workspace_name = "test_workspace"
datastore_name = "test_datastore"
geoserver_datastore_path = "/geoserver-test-data"
shapefile_name = "Espace_Vert"

print("  Verify SSL : {}".format(VERIFY_SSL))
print("  Will use Magpie URL:   [{}]".format(MAGPIE_URL))
print("  Will use Geoserver secured URL:   [{}]".format(GEOSERVER_SECURED_URL))
print("  Will use Geoserver admin rest URL:  [{}]".format(GEOSERVER_ADMIN_URL))

In [ ]:
def magpie_signin(user_name, password):
    signin_url = f"{MAGPIE_URL}/signin"
    data = {"user_name": user_name, "password": password}
    try:
        resp = requests.request(url=signin_url, method="POST", data=data, timeout=10, verify=VERIFY_SSL)
    except Exception as exc:
        raise RuntimeError(f"Failed to sign in to Magpie (url: `{signin_url}`) with user `{data['user_name']}`. "
                           f"Exception : {exc}. ")
    if resp.status_code != 200:
        raise RuntimeError(f"Unexpected response while trying to sign in to Magpie : {resp.text}")
    return resp

In [ ]:
geoserver_admin_session = requests.Session()
geoserver_admin_session.verify = VERIFY_SSL
geoserver_admin_session.headers = HEADERS
geoserver_admin_session.auth = (TEST_GEOSERVER_ADMIN_USERNAME, TEST_GEOSERVER_ADMIN_PASSWORD)

magpie_admin_session = requests.Session()
magpie_admin_session.verify = VERIFY_SSL
magpie_admin_session.headers = HEADERS
magpie_admin_session.cookies = magpie_signin(TEST_MAGPIE_ADMIN_USERNAME, TEST_MAGPIE_ADMIN_PASSWORD).cookies

test_user_session = requests.Session()
test_user_session.verify = VERIFY_SSL
test_user_session.headers = HEADERS

no_perm_user_session = requests.Session()
no_perm_user_session.verify = VERIFY_SSL
no_perm_user_session.headers = HEADERS

In [ ]:
def response_msg(message, response, is_json=True):
    """Append useful response details to provided message."""
    _body = response.text
    _detail = "<unknown>"
    if is_json:
        try:
            _body = response.json()
            _detail = _body.get("detail", _body.get("message", "<unknown>"))
        except json.JSONDecodeError:
            _body = response.text
            pass  # ignore and revert to text body since it could not be parsed as JSON
    return "{} Response replied with ({}) [{}]\nContent: {}\n\n".format(message, response.status_code, _detail, _body)

In [ ]:
CLEANUP_CALLED = False

def cleanup_test_data(skip_fail=True):
    """Cleanup test elements created during the test.

    Only raise at the complete end to attempt removal of as much element as possible in case of failures.
    """
    global CLEANUP_CALLED

    CLEANUP_CALLED = True
    failed = False
    # Cleanup Magpie users
    for item_type, item_name, item_value in [
        ("users", "MAGPIE_TEST_USER", MAGPIE_TEST_USER),
        ("users", "MAGPIE_NO_PERM_USER", MAGPIE_NO_PERM_USER),
    ]:
        try:
            _path = "{}/{}/{}".format(MAGPIE_URL, item_type, item_value)
            _resp = magpie_admin_session.delete(_path)
            _head = "Cleanup of {} [{}]:".format(item_name, item_value)
            if _resp.status_code == 200:
                print(_head, "OK")
            elif _resp.status_code == 404:
                print(_head, "WARNING - does not exist (maybe it failed to be created in the first place?)")
                failed = True  # most definitely the test is already failing at this point anyway
            else:
                print(_head, "ERROR - could not be removed")
                failed = True
        except Exception as exc:
            print("Magpie cleanup raised: [{}]".format(exc))
            failed = True
    
    # Cleanup Geoserver workspace (must remove each resource step by step)
    for path in [
        f"{GEOSERVER_ADMIN_URL}/layers/{shapefile_name}",
        f"{GEOSERVER_ADMIN_URL}/workspaces/{workspace_name}/datastores/{datastore_name}/featuretypes/{shapefile_name}",
        f"{GEOSERVER_ADMIN_URL}/workspaces/{workspace_name}/datastores/{datastore_name}",
        f"{GEOSERVER_ADMIN_URL}/workspaces/{workspace_name}"
    ]:
        try:
            _resp = geoserver_admin_session.delete(url=path)
            _head = "Cleanup of [{}]:".format(path)
            if _resp.status_code == 200:
                print(_head, "OK")
            elif _resp.status_code == 404:
                print(_head, "WARNING - does not exist (maybe it failed to be created in the first place?)")
                failed = True  # most definitely the test is already failing at this point anyway
            else:
                print(_head, "ERROR - could not be removed")
                failed = True
        except Exception as exc:
            print("Geoserver cleanup raised: [{}]".format(exc))
            failed = True
    
    if not skip_fail and failed:
        raise ValueError("Could not cleanup all test data.")

### Prepare Geoserver data

In [ ]:
# create workspace
payload = {"workspace": {"name": workspace_name, "isolated": "True"}}
resp = geoserver_admin_session.post(url=f"{GEOSERVER_ADMIN_URL}/workspaces", json=payload)

if resp.status_code != 201:
    cleanup_test_data()
    raise ValueError(response_msg("\nFailed to create Geoserver workspace `{}`.".format(workspace_name), resp))

In [ ]:
# create datastore
payload = {
    "dataStore": {
        "name": datastore_name,
        "type": "Directory of spatial files (shapefiles)",
        "connectionParameters": {
            "entry": []
        }}}
resp = geoserver_admin_session.post(url=f"{GEOSERVER_ADMIN_URL}/workspaces/{workspace_name}/datastores", json=payload)

if resp.status_code != 201:
    cleanup_test_data()
    raise ValueError(response_msg("\nFailed to create Geoserver datastore `{}`.".format(datastore_name), resp))

In [ ]:
# configure datastore
payload = {
    "dataStore": {
        "name": datastore_name,
        "type": "Directory of spatial files (shapefiles)",
        "connectionParameters": {
            "entry": [
                {"$": "UTF-8",
                 "@key": "charset"},
                {"$": "shapefile",
                 "@key": "filetype"},
                {"$": "true",
                 "@key": "create spatial index"},
                {"$": "true",
                 "@key": "memory mapped buffer"},
                {"$": "GMT",
                 "@key": "timezone"},
                {"$": "true",
                 "@key": "enable spatial index"},
                {"$": f"http://{datastore_name}",
                 "@key": "namespace"},
                {"$": "true",
                 "@key": "cache and reuse memory maps"},
                {"$": f"file://{geoserver_datastore_path}",
                 "@key": "url"},
                {"$": "shape",
                 "@key": "fstype"},
            ]
        },
    }
}
geoserver_admin_session.put(url=f"{GEOSERVER_ADMIN_URL}/workspaces/{workspace_name}/datastores/{datastore_name}", json=payload)

if resp.status_code != 201:
    cleanup_test_data()
    raise ValueError(response_msg("\nFailed to configure Geoserver datastore `{}`.".format(datastore_name), resp))

In [ ]:
# publish shapefile
shapefile_payload = {
    "featureType": {
        "name": shapefile_name,
        "nativeCRS": """
                        GEOGCS[
                            "WGS 84", 
                            DATUM[
                                "World Geodetic System 1984",
                                SPHEROID["WGS 84", 6378137.0, 298.257223563, AUTHORITY["EPSG","7030"]],
                                AUTHORITY["EPSG","6326"]
                            ],
                            PRIMEM["Greenwich", 0.0, AUTHORITY["EPSG","8901"]],
                            UNIT["degree", 0.017453292519943295],
                            AXIS["Geodetic longitude", EAST],
                            AXIS["Geodetic latitude", NORTH],
                            AUTHORITY["EPSG","4326"]
                        ]
                    """,
        "srs": "EPSG:4326",
        "projectionPolicy": "REPROJECT_TO_DECLARED",
        "maxFeatures": 5000,
        "numDecimals": 6,
    }
}

resp = geoserver_admin_session.post(
    url=f"{GEOSERVER_ADMIN_URL}/workspaces/{workspace_name}/datastores/{datastore_name}/featuretypes",
    json=shapefile_payload
)

if resp.status_code != 201:
    cleanup_test_data()
    raise ValueError(response_msg("\nFailed to publish shapefile `{}` to Geoserver.".format(shapefile_name), resp))

### Prepare Magpie users and resources

In [ ]:
def create_magpie_user(user_name, password, session):
    resp = magpie_admin_session.post(
        url=f"{MAGPIE_URL}/users",
        json={
            "user_name": user_name,
            "email": f"{user_name}@user.com",
            "password": password,
            "group_name": "users"},
        allow_redirects=False)
    if resp.status_code != 201:
        cleanup_test_data()
        raise ValueError(response_msg("\nCould not create test user [{}]".format(user_name), resp))
    session.cookies = magpie_signin(user_name, password).cookies

create_magpie_user(MAGPIE_TEST_USER, MAGPIE_TEST_PASSWORD, test_user_session)
create_magpie_user(MAGPIE_NO_PERM_USER, MAGPIE_NO_PERM_PASSWORD, no_perm_user_session)

In [ ]:
# create/get resources
resp = magpie_admin_session.get(f"{MAGPIE_URL}/services/{GEOSERVER_SECURED_SERVICE}")
if resp.status_code != 200:
    cleanup_test_data()
    raise ValueError(response_msg("\nCould not fetch secured Geoserver service details [{}].".format(GEOSERVER_SECURED_SERVICE), resp))
geoserver_res_id = resp.json()["service"]["resource_id"]

def create_or_get_resource(resource_name, resource_type, parent_id):
    _path = MAGPIE_URL + "/resources"
    _data = {"parent_id": parent_id, "resource_type": resource_type, "resource_name": resource_name}
    _resp = magpie_admin_session.post(_path, json=_data)
    _msg = "[name: {}, type: {}, parent: {}].".format(resource_name, resource_type, parent_id)
    if _resp.status_code == 409:
        _path = "{}/{}".format(_path, parent_id)
        _resp = magpie_admin_session.get(_path)
        if _resp.status_code != 200:
            cleanup_test_data()
            raise ValueError(response_msg("Could not retrieve resource expected to exist {}.".format(_msg), _resp))
        _resource = _resp.json()["resource"]
        _children = _resource["children"]
        for _, _child_info in _children.items():
            if _child_info["resource_name"] == resource_name:
                return _child_info
    elif _resp.status_code != 201:
        cleanup_test_data()
        raise ValueError(response_msg("\nCould not create resource {}.".format(_msg), _resp))
    else:
        return _resp.json()["resource"]
    cleanup_test_data()
    raise ValueError(response_msg("\nCould not create or retrieve resource {}.".format(_msg), _resp))

workspace_res = create_or_get_resource(workspace_name, "workspace", geoserver_res_id)
layer_res = create_or_get_resource(shapefile_name, "layer", workspace_res["resource_id"])

In [ ]:
def set_permission(permission, resource, target_type, target_name):
    _res_id = resource["resource_id"]
    _path = "{}/{}/{}/resources/{}/permissions".format(MAGPIE_URL, target_type, target_name, _res_id)
    _data = {"permission": permission}
    _resp = magpie_admin_session.put(_path, json=_data)
    if _resp.status_code not in [200, 201]:
        _msg = "\nCleanup called before? {} (if called, following error could be expected)".format(CLEANUP_CALLED) + \
               "\nCould not set permission [{}] for [{}, {}] over resource [{}, {}]" \
               .format(permission, target_type, target_name, resource["resource_name"], _res_id)
        cleanup_test_data()
        raise ValueError(response_msg(_msg, _resp))

### Test user access

In [ ]:
def has_access(path, session, user_name):
    _resp = session.get(path, headers = {"Content-Type": None})
    _code = _resp.status_code
    if _code in [200, 401, 403]:
        is_allowed = _code == 200
        str_allowed = "Allowed" if is_allowed else "Denied"
        print("Detail:\n"
              "  Resource: [{}]\n"
              "  User:     [{}]\n"
              "  Code:     [{}]\n"
              "  Access:   [{}]".format(path, user_name, _code, str_allowed))
        return is_allowed
    cleanup_test_data()
    print("Detail:\n"
          "  Resource: [{}]\n"
          "  User:     [{}]\n"
          "  Code:     [{}]\n"
          "  Access:   [Error]".format(path, user_name, _code))
    _msg = "Unexpected status code during access to resource [{}]".format(path)
    raise ValueError(response_msg(_msg, _resp, is_json=False))

In [ ]:
# Test on workspace resource
res_path = f"{GEOSERVER_SECURED_URL}/{workspace_name}/wfs?typeNames={workspace_name}:{shapefile_name}&request=GetFeature"

assert not has_access(res_path, test_user_session, MAGPIE_TEST_USER)
assert not has_access(res_path, no_perm_user_session, MAGPIE_NO_PERM_USER)

set_permission("getfeature-allow-recursive", workspace_res, "users", MAGPIE_TEST_USER)

assert has_access(res_path, test_user_session, MAGPIE_TEST_USER)
assert not has_access(res_path, no_perm_user_session, MAGPIE_NO_PERM_USER)

In [ ]:
# Test on layer resource
res_path = f"{GEOSERVER_SECURED_URL}/{workspace_name}/wfs?typeNames={workspace_name}:{shapefile_name}&request=DescribeStoredQueries"

assert not has_access(res_path, test_user_session, MAGPIE_TEST_USER)
assert not has_access(res_path, no_perm_user_session, MAGPIE_NO_PERM_USER)

set_permission("describestoredqueries-allow-match", layer_res, "users", MAGPIE_TEST_USER)

assert has_access(res_path, test_user_session, MAGPIE_TEST_USER)
assert not has_access(res_path, no_perm_user_session, MAGPIE_NO_PERM_USER)

In [ ]:
# final cleanup, everything is expected to have worked up to here,
# so force failure to ensure we return to original 'clean' state
cleanup_test_data(skip_fail=False)
print("All tests: OK")